In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import os
import gmaps
import seaborn as sn
import re

# Import API key from a file that is ignored by Git (.gitignore file) so the key isn't exposed to the public
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

## Store County Health Rankings Excel file results into DataFrame

* Load the excel file imported from https://www.countyhealthrankings.org/app/texas/2019/measure/outcomes/144/description?sort=desc-2

In [2]:
path = "data/2019 County Health Rankings Texas Data - v1_0.xls"

# Get the available sheets in the excel file and put into a list
sheets = pd.ExcelFile(path).sheet_names
print(sheets)

['Introduction', 'Outcomes & Factors Rankings', 'Outcomes & Factors SubRankings', 'Ranked Measure Data', 'Additional Measure Data', 'Ranked Measure Sources & Years', 'Addtl Measure Sources & Years']


## Read in the Ranked Measure Data sheet eliminating columns and extra headers
* Make a list of available columns to decide what we don't want
* Formulate a regex expression to match unwanted columns
* Create a list of only wanted columns to be used for the usecols argument
* Set county to be the index
* Eliminate any row without a county name

In [3]:
# Get the 'Ranked Measure Data' sheet using the sheets list above and rejecting the top header row
rmd = pd.read_excel(path, header=[1], sheet_name=sheets[3])

# View the columns and decide what we don't want
[print(col) for col in rmd.columns] 

FIPS
State
County
Years of Potential Life Lost Rate
95% CI - Low
95% CI - High
Z-Score
YPLL Rate (Black)
YPLL Rate (Hispanic)
YPLL Rate (White)
% Fair/Poor
95% CI - Low.1
95% CI - High.1
Z-Score.1
Physically Unhealthy Days
95% CI - Low.2
95% CI - High.2
Z-Score.2
Mentally Unhealthy Days
95% CI - Low.3
95% CI - High.3
Z-Score.3
Unreliable
% LBW
95% CI - Low.4
95% CI - High.4
Z-Score.4
% LBW (Black)
% LBW (Hispanic)
% LBW (White)
% Smokers
95% CI - Low.5
95% CI - High.5
Z-Score.5
% Obese
95% CI - Low.6
95% CI - High.6
Z-Score.6
Food Environment Index
Z-Score.7
% Physically Inactive
95% CI - Low.7
95% CI - High.7
Z-Score.8
% With Access
Z-Score.9
% Excessive Drinking
95% CI - Low.8
95% CI - High.8
Z-Score.10
# Alcohol-Impaired Driving Deaths
# Driving Deaths
% Alcohol-Impaired
95% CI - Low.9
95% CI - High.9
Z-Score.11
# Chlamydia Cases
Chlamydia Rate
Z-Score.12
Teen Birth Rate
95% CI - Low.10
95% CI - High.10
Z-Score.13
Teen Birth Rate (Black)
Teen Birth Rate (Hispanic)
Teen Birth Rate (W

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [4]:
# Create a regular expression to match unwanted columns
regex = '^Z|95|FIPS|State|Unreliable'

# Use a list comprehension to make a list of columns that don't match the regex (a failed regex match returns 'None')
cols = [col for col in rmd.columns if re.match(regex,col) is None]

# Create a dataframe of the desired columns and set 'County' to the index
rmd_df = pd.read_excel(path, header=[1], sheet_name=sheets[3],usecols=cols,index_col=0)

# Drop any row that has Nan as an index value
rmd_df = rmd_df.loc[rmd_df.index.dropna()]
print(f'shape of rmd_df: {rmd_df.shape}')
#rmd_df.head(1)

shape of rmd_df: (254, 85)


## Read in the Additional Measure Data sheet eliminating columns and extra headers
* Make a list of available columns to decide what we don't want
* Formulate a regex expression to match unwanted columns
* Create a list of only wanted columns to be used for the usecols argument
* Set county to be the index
* Eliminate any row without a county name

In [5]:
# Get the 'Ranked Measure Data' sheet using the sheets list above and rejecting the top header row
amd = pd.read_excel(path, header=[1], sheet_name=sheets[4])

# View the columns and decide what we don't want
[print(col) for col in amd.columns] 

FIPS
State
County
Life Expectancy
95% CI - Low
95% CI - High
Life Expectancy (Black)
Life Expectancy (Hispanic)
Life Expectancy (White)
# Deaths
Age-Adjusted Mortality
95% CI - Low.1
95% CI - High.1
Age-Adjusted Mortality (Black)
Age-Adjusted Mortality (Hispanic)
Age-Adjusted Mortality (White)
# Deaths.1
Child Mortality Rate
95% CI - Low.2
95% CI - High.2
Child Mortality Rate (Black)
Child Mortality Rate (Hispanic)
Child Mortality Rate (White)
# Deaths.2
Infant Mortality Rate
95% CI - Low.3
95% CI - High.3
Infant Mortality Rate (Black)
Infant Mortality Rate (Hispanic)
Infant Mortality Rate (White)
% Frequent Physical Distress
95% CI - Low.4
95% CI - High.4
% Frequent Mental Distress
95% CI - Low.5
95% CI - High.5
% Diabetic
95% CI - Low.6
95% CI - High.6
# HIV Cases
HIV Prevalence Rate
# Food Insecure
% Food Insecure
# Limited Access
% Limited Access
# Drug Overdose Deaths
Drug Overdose Mortality Rate
# Motor Vehicle Deaths
MV Mortality Rate
95% CI - Low.7
95% CI - High.7
% Insufficien

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
# Create a regular expression to match unwanted columns
regex = '^Z|95|FIPS|State|Unreliable'

# Use a list comprehension to make a list of columns that don't match the regex (a failed regex match returns 'None')
cols = [col for col in amd.columns if re.match(regex,col) is None]

# Create a dataframe of the desired columns and set 'County' to the index
amd_df = pd.read_excel(path, header=[1], sheet_name=sheets[4],usecols=cols,index_col=0)

# Drop any row that has Nan as an index value
amd_df = amd_df.loc[amd_df.index.dropna()]
print(f'shape of amd_df: {amd_df.shape}')
#amd_df.head(1)

shape of amd_df: (254, 74)


In [7]:
# Merge the rmd and amd dataframes using inner join on County column (index)
merged_df = pd.merge(rmd_df,amd_df, how='inner', on='County')
print(f'shape of merged_df: {merged_df.shape}')
#merged_df.head(1)

shape of merged_df: (254, 159)


In [16]:
# View the columns that end in '_x' or '_y'
regex2 = '.*_x$|.*_y$'
[print(col) for col in merged_df.columns if re.match(regex2,col) is not None] 

# Uninsured_x
% Uninsured_x
Population_x
# Uninsured_y
% Uninsured_y
Population_y


[None, None, None, None, None, None]

In [17]:
# Drop the duplicate _y columns
regex3 = '.*_y$'
merged_df.drop([col for col in merged_df.columns if re.match(regex3,col) is not None],axis=1, inplace=True)
print(merged_df.shape)

(254, 156)
